# Modeling
See 'Notebooks/Data_Cleaning.ipynb' for data cleaning. Raw data imported from square.com was cleaned and some relevant features impacting the sales, such as weather and social media, were added.

- [Visualizations](#Visualizations)
- [Recursive Feature Elimination](#Recursive-Feature-Elimination)
- [Model](#Model)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns


from sklearn.base import clone
from sklearn.model_selection import train_test_split

from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler


from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score

from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

from random import randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn import set_config


In [ ]:
sales_data = pd.read_csv('insert_path')

In [ ]:
sales_data = sales_data.drop('Unnamed: 0', axis='columns')

In [ ]:
# Remove any non-beer transactions
sales_data = sales_data[sales_data['beer_style'] != 'None']

# Remove any cash transactions as the customers cannot be tracked
sales_data = sales_data[sales_data['customer_id_no'] != -1]

# Remove unnecessary columns
sales_data = sales_data.drop(columns = {'item','customer_init','transaction_id'})
#Change types
sales_data['daily_no_customers'] = sales_data['daily_no_customers'].astype(int)
sales_data['no_styles_sold'] = sales_data['no_styles_sold'].astype(int)

In [ ]:
bool_dict = {'No Post' : 0,
             'Beer' : 0,
             'Event' : 1,
             'None' : 0}
bool_dict_weekday = {'Monday' : 0,
                     'Tuesday' : 0,
                     'Wednesday' : 0,
                     'Thursday' : 0,
                     'Friday': 1,
                     'Saturday': 1,
                     'Sunday': 1}

sales_data['insta_post_type'].replace(bool_dict, inplace=True)
sales_data['weekday'].replace(bool_dict_weekday, inplace=True)
sales_data = sales_data.rename(columns={'insta_post_type':'event_promoted',
                                        'weekday':'is_weekend'})
sales_data.loc[sales_data['precipitation'] > 0, 'precipitation'] = 1 # is there precipitation?
sales_data.loc[sales_data['snowfall'] > 0, 'snowfall'] = 1 # is there snow?

sales_data['avg_temp'] = sales_data[['max_temp', 'min_temp']].mean(axis=1)

In [ ]:
sales_data['can_purchase'] = sales_data['product_type']
sales_data['flight_purchase'] = sales_data['product_type']
sales_data['draft_purchase'] = sales_data['product_type']
sales_data['first_visit_year'] = sales_data['year']

In [ ]:
# group by transaction to get each transaction into a single line
temp = sales_data.groupby('transaction_no').agg({'customer_id_no':'max',
                                                 'gross_sales':'sum',
                                                 'year':'max',
                                                 'first_visit_year':'min',
                                                 'beer_style':lambda x: x.nunique(),
                                                 'can_purchase':lambda x: 1 if 'Can' in x.values else 0,
                                                 'flight_purchase':lambda x: 1 if 'Flight' in x.values else 0,
                                                 'draft_purchase':lambda x: 1 if 'Draft' in x.values else 0,
                                                 'visit_freq':'max',
                                                 'no_styles_sold':'mean',
                                                 'primary_beer_type':lambda x: x.value_counts().idxmax(),
                                                 'beer_abv':'mean',
                                                 'global_rating':'mean',
                                                 'insta_post':'max',
                                                 'event_promoted':'max',
                                                 'daily_no_customers':'max',
                                                 'avg_temp':'max'})

In [ ]:
# group by customer to get data for each trackable customer
customers = temp.groupby('customer_id_no').agg({'gross_sales':'max',
                                                'year':lambda x: x.nunique(),
                                                'first_visit_year':'min',
                                                'beer_style':'max',
                                                'can_purchase':'max',
                                                'flight_purchase':'max',
                                                'draft_purchase':'max',
                                                'visit_freq':'max',
                                                'no_styles_sold':'mean',
                                                'primary_beer_type':lambda x: x.value_counts().idxmax(),
                                                'daily_no_customers':'max',
                                                'avg_temp':'max',
                                                'beer_abv':'mean',
                                                'global_rating':'mean',
                                                'insta_post':'max',
                                                'event_promoted':'max'})

customers = pd.DataFrame(customers).reset_index()
customers = customers[~(customers['customer_id_no'] == -1)]

customers = customers.rename(columns={'beer_style':'beer_style_count',
                                      'event_promoted':'attended_event'})

customers['returning'] = customers['visit_freq']

customers['returning'] = customers['returning'].apply(lambda x: 'Return' if x > 1 else 'Lost')

# Visualizations

In [ ]:
daily_returns = sales_data[['date','visit_freq']]
daily_returns['returning'] = daily_returns['visit_freq']
daily_returns['returning'] = daily_returns['returning'].apply(lambda x: 'Return' if x > 1 else 'Lost')

daily_returner = daily_returns.groupby('date').returning.value_counts().unstack(fill_value=0)
daily_returner['percent_returned'] = daily_returner['Return'] / (daily_returner['Return'] + daily_returner['Lost'])

In [ ]:
percent_return = customers['returning'].value_counts()['Return']/(customers['returning'].value_counts()['Lost'] + customers['returning'].value_counts()['Return'])
percent_lost = customers['returning'].value_counts()['Lost']/(customers['returning'].value_counts()['Lost'] + customers['returning'].value_counts()['Return'])

print(f"Percentage of customers returning: {percent_return:.0%}" f"\nPercentage of customers lost: {percent_lost:.0%}")
print(f"Percentage of daily customers that are returning customers: {(daily_returner['percent_returned'].mean()):.0%}")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

daily_visitors = sales_data.groupby('date').agg({'daily_no_customers':'max',
                                                 'year':'max',
                                                 'month':'max',
                                                 'no_styles_sold':'max',
                                                 'gross_sales': 'sum',
                                                 'global_rating': 'mean'}).reset_index()
daily_visitors.set_index('date', inplace=True)
daily_visitors = daily_visitors[daily_visitors['year'] > 2019]
daily_visitors_monthly = daily_visitors['daily_no_customers'].resample('M').mean().reset_index()

sns.set_style("whitegrid")
plt.figure(figsize=(12, 6))
sns.lineplot(data=daily_visitors_monthly, x='date', y='daily_no_customers')
plt.title('Average Monthly Customers from 2020 to Present')
plt.xlabel('Date')
plt.ylabel('Customer Count')
plt.show()

In [ ]:
daily_visitors_plt = daily_visitors.groupby('year')['daily_no_customers'].mean()
daily_visitors_plt = pd.DataFrame(daily_visitors_plt).reset_index()

palette = "colorblind"
sns.barplot(data=daily_visitors_plt, x='year', y='daily_no_customers', palette=palette)
plt.title('Daily Customers Average per Year')
plt.xlabel('Year')
plt.ylabel('Average Daily Customers')
plt.show()

In [ ]:
palette = sns.color_palette("colorblind")
sns.countplot(data=customers, x = 'returning', hue='returning', palette=palette)
plt.title('Lost and Returning Customer Counts from 2020 to Present', fontsize=20)
plt.xlabel('Customer status', fontsize=16)
plt.ylabel('Count of customers', fontsize=16)
plt.show()

In [ ]:
# correlation plot
customers_plot = customers[customers['first_visit_year'] < 2023]
import numpy as np
customers_dropped = customers_plot.drop(columns={'customer_id_no',
                                                 'first_visit_year',
                                                 'primary_beer_type',
                                                 'visit_freq',
                                                 'year'}, axis=1)

customers_dropped['returning'] = customers_dropped['returning'].apply(lambda x: 1 if x == 'Return' else 0)

customers_dropped_corr = customers_dropped.corrwith(customers_dropped['returning'])
y_1 = customers_dropped_corr.index
y_1 = y_1[:-1]
x_1 = ['returning']
sns.heatmap(customers_dropped_corr[:-1,np.newaxis], annot = True, xticklabels = x_1, yticklabels = y_1);

# Recursive Feature Elimination

In [ ]:
# creation of X,y, training and testing
# encoding
X = customers.drop(['returning','customer_id_no','year','draft_purchase','visit_freq'],axis=1)

y = customers['returning']
ohe_cols = X.select_dtypes('object').columns.tolist()

transformer = make_column_transformer((OneHotEncoder(), ohe_cols),
                                      remainder = StandardScaler())

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.2, random_state=14)

training = pd.DataFrame(transformer.fit_transform(X_train), columns = transformer.get_feature_names_out())
training.head()

In [ ]:
print("Features: \n mean:\n {}\n standard deviation: \n{}".format(training.mean(), training.std()))

- the above shows the scaler is working correctly: mean approx 0 and stdev is approx 1

## Determine number of features

In [ ]:
score_val = []
count_val = []
for i in range(1, len(X.columns)):
    rfe = RFE(estimator=LogisticRegression(), n_features_to_select=i)
    pipe = Pipeline([('encoder', transformer),
     ('selector', rfe),
     ('model', LogisticRegression(solver = 'lbfgs', max_iter = 1000, random_state=14))])
    pipe.fit(X_train, y_train)
    pipe.score(X_train, y_train)

    count_val.append(i)
    score_val.append(pipe.score(X_train, y_train))

In [ ]:
plt.plot(count_val, score_val)
plt.xticks(list(range(1, 12,1)))
plt.xlabel('Number of features')
plt.ylabel('Score')
plt.ylabel('Score for number of features')
plt.title('Determination of number of features')
plt.show()

- Revise number of features below based on the above

In [ ]:
n_feat = 4

rfe = RFE(estimator=LogisticRegression(), n_features_to_select=n_feat)
pipe = Pipeline([('encoder', transformer),
 ('selector', rfe),
 ('logreg', LogisticRegression(solver = 'lbfgs', max_iter = 1000, random_state=14))])
pipe.fit(X_train, y_train)

In [ ]:
ranking_log = pipe.named_steps['selector'].ranking_

In [ ]:
print('Train score: ' + str(pipe.score(X_train, y_train)))
print('Test score: ' + str(pipe.score(X_test, y_test)))
top_feat_dict = str(sorted(zip(map(lambda x: round(x,4), ranking_log), X.columns)))
print('Top features: ' + top_feat_dict)
from sklearn.metrics import accuracy_score
predictions_log = pipe.predict(X_test)
accuracy_log = accuracy_score(y_test, predictions_log)
print('Logistic regression accuracy: {:.0%}'.format(accuracy_log))

- Revise chosen features based on the above

In [ ]:
imp_feat = customers[['global_rating','no_styles_sold','beer_style_count','can_purchase']]

In [ ]:
cm_log = confusion_matrix(y_test, predictions_log, labels=pipe.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_log, display_labels=pipe.classes_)
disp.plot()
plt.show()

# Model
- Build a model off of the chosen features

In [ ]:
X_new = imp_feat

y_new = customers['returning']
ohe_cols = X_new.select_dtypes('object').columns.tolist()

transformer = make_column_transformer((OneHotEncoder(), ohe_cols),
                                      remainder = StandardScaler())

X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_new, y_new,
                                                        test_size = 0.2, random_state=14)

In [ ]:
models = [LogisticRegression(), LinearSVC(), DecisionTreeClassifier(),
          KNeighborsClassifier(), GaussianNB(), RandomForestClassifier()]

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import classification_report

import math

model_dict = {}
model_dict_2 = {}

for model in models:
    scores=[]
    pipe = Pipeline([('encoder', transformer), ('model', model)])
    pipe.fit(X_train_new, y_train_new)
    prediction = model.predict(X_test_new) 
    scores.append(cross_val_score(pipe, X_new, y_new, cv=5))
    model_dict[str(model)] = scores

In [ ]:
scores_df = pd.DataFrame.from_dict(model_dict, orient='index')
scores_df['0'] = scores_df[0].apply(lambda x: x[0])
scores_df['1'] = scores_df[0].apply(lambda x: x[1])
scores_df['2'] = scores_df[0].apply(lambda x: x[2])
scores_df['3'] = scores_df[0].apply(lambda x: x[3])
scores_df['4'] = scores_df[0].apply(lambda x: x[4])
scores_df = scores_df.drop(columns={0}, axis = 1)
scores_df = scores_df.transpose()
scores_df.head()

- Based on the scores above, choose the best model - here Random Forest was used

In [ ]:
param_dist = {'n_estimators': range(1,500,50),
              'max_depth': range(1,20,1)}

rf = RandomForestClassifier()

# Use random search to find the best hyperparameters
rand_search = RandomizedSearchCV(rf, 
                                 param_distributions = param_dist, 
                                 n_iter=5, 
                                 cv=5,
                                random_state=14)

# Fit the random search object to the data
rand_search.fit(X_train_new, y_train_new)

# Create a variable for the best model
best_rf = rand_search.best_estimator_

# Print the best hyperparameters
print('Best hyperparameters:',  rand_search.best_params_)

best_rf

In [ ]:
# Choosing random forest
set_config(transform_output="pandas")

pipe_rf = Pipeline([('encoder', transformer), ('model', best_rf)])
pipe_rf.fit(X_train_new, y_train_new)

In [ ]:
predictions_rf = pipe_rf.predict(X_test_new)
accuracy_rf = accuracy_score(y_test_new, predictions_rf)
print('Accuracy score {:.0%}'.format(accuracy_rf))

In [ ]:
cm = confusion_matrix(y_test_new, predictions_rf, labels=pipe_rf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=pipe_rf.classes_)
disp.plot()
plt.title('Confusion Matrix')
plt.show()

In [ ]:
RocCurveDisplay.from_estimator(pipe_rf, X_test_new, y_test_new)
plt.plot([0, 1], [0, 1])
sns.set_style("whitegrid")
plt.title('Receiver operating characteristic (ROC) curve')
plt.show()

In [ ]:
tp = cm[0][0]
fp = cm[0][1]
tn = cm[1][1]
fn = cm[1][0]
accuracy = (tp + tn) / (tp + fp + tn + fn)
error_rate = 1 - accuracy
true_pos_rate = tp / (tp +fn)
false_pos_rate = fn / (tp +fn)
print("Model accuracy: {:.2}".format(accuracy))
print("Model error rate: {:.2}".format(error_rate))
print("Model true positive rate: {:.2}".format(true_pos_rate))
print("Model false positive rate: {:.2}".format(false_pos_rate))

- Verify that the confusion matrix is in line with the accuracy score.
- A more perfect model would have the ROC curve close to the y-axis and 1.0, giving an AUC of 1.

In [ ]:
print(pd.Series(predictions_rf[:10]))
print(y_test_new[:10].reset_index().drop('index',axis=1))

In [ ]:
rf = pipe_rf[-1]
data = list(zip(rf.feature_names_in_, rf.feature_importances_))
df_importances = pd.DataFrame(data, columns=['Feature', 'Importance']).sort_values(by='Importance', ascending=False)
df_importances['Feature'] = df_importances['Feature'].replace("remainder__", '', regex=True)

In [ ]:
sns.barplot(data = df_importances, y='Feature', x='Importance', palette=palette)
plt.title("Feature Importance")
plt.show()

- Following determination of most important features, dig deeper and look for actionable solutions based on those features. In the case of my project, I found customers who return try 1 more beer on average per visit, so it may make sense to push new customers towards flights. The purchasing of cans could also predict a customer's return, a can fridge had been installed recently and the effects are yet to be determined. If a lot of styles were available, customers were more likely to return which would explain the drop of returning customers over time that this brewery was experiencing. Investments in new styles were considered.